<a href="https://colab.research.google.com/github/acse-srm3018/ReservoirClassification/blob/main/src_python/reservoir_classification_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import scipy.io
mat = scipy.io.loadmat('data.mat')

In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

In [8]:
print(mat)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Mar 06 10:27:42 2018', '__version__': '1.0', '__globals__': [], 'data': array([[-8.07622998e+01, -8.07132045e+01, -7.64687027e+01, ...,
        -1.03233827e+01, -8.75157039e+00, -7.64773376e+00],
       [-9.73119201e+01, -9.76039926e+01, -9.04719387e+01, ...,
        -7.17149926e+00, -5.89261733e+00, -5.03178911e+00],
       [-1.14006359e+02, -1.15001008e+02, -1.03566971e+02, ...,
        -5.20768799e+00, -4.25234451e+00, -3.62696439e+00],
       ...,
       [ 1.00000000e-02,  1.00000000e-02,  1.00000000e-02, ...,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02],
       [ 1.00000000e-02,  1.00000000e-02,  1.00000000e-02, ...,
         1.00000000e-02,  1.00000000e-02,  1.00000000e-02],
       [ 1.00000000e-02,  1.00000000e-02,  1.00000000e-02, ...,
         9.90000000e-01,  9.90000000e-01,  9.90000000e-01]])}


In [20]:
type(mat)

dict

In [21]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data'])

In [23]:
type(mat['data']),mat['data'].shape

(numpy.ndarray, (54, 2554))

In [24]:
type(mat['data'][0][0]),mat['data'][0][0].shape

(numpy.float64, ())

In [26]:
print(mat['data'][0])

[-80.76229978 -80.71320452 -76.46870269 ... -10.32338272  -8.75157039
  -7.64773376]


In [28]:
mat['data'][0][0], mat['data'][0][0]

(-80.762299781504, -80.762299781504)

In [33]:
#data = [[row.flat[0] for row in line] for line in mat['data'][0]]
df_train = pd.DataFrame(mat['data'])

In [34]:
df_train.head


<bound method NDFrame.head of           0           1           2     ...        2551        2552        2553
0   -80.762300  -80.713205  -76.468703  ...  -10.323383   -8.751570   -7.647734
1   -97.311920  -97.603993  -90.471939  ...   -7.171499   -5.892617   -5.031789
2  -114.006359 -115.001008 -103.566971  ...   -5.207688   -4.252345   -3.626964
3  -128.915758 -131.040086 -113.771098  ...   -4.111738   -3.403200   -2.941437
4  -139.180704 -142.857405 -118.507537  ...   -3.543173   -2.988997   -2.622225
5  -142.133108 -147.629407 -115.854635  ...   -3.242124   -2.774154   -2.458039
6  -135.912053 -143.173981 -105.186503  ...   -3.061337   -2.641134   -2.352919
7  -120.542290 -129.056750  -87.896717  ...   -2.944865   -2.552817   -2.281347
8   -98.547786 -107.364346  -67.348302  ...   -2.865779   -2.491424   -2.230695
9   -74.381153  -82.397125  -47.660604  ...   -2.811973   -2.449013   -2.195263
10  -52.655505  -59.028074  -31.986299  ...   -2.791301   -2.434516   -2.184332
11  -36.23

In [39]:
train = np.array(mat['data'])

In [42]:
train.shape

(54, 2554)

In [45]:
train_x=train[0:48,:]
train_y=train[48:54,:]

In [47]:
train_y.shape

(6, 2554)

In [54]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
scaled_x = scaler.fit_transform(train_x)
scaled_y = scaler.fit_transform(train_y)

In [51]:
%pylab inline
import torch
import torch.nn as nn

def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  # uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. Useful when inputs do not change size -
    torch.backends.cudnn.enabled   = False

    return True

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['mat']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.transpose(scaled_x), np.transpose(scaled_y), test_size=0.20, random_state=42)

In [57]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [66]:
input_layer = Input(shape=(np.transpose(scaled_x).shape[1],))
dense_layer_1 = Dense(20, activation='relu')(input_layer)
dense_layer_2 = Dense(15, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(10, activation='relu')(dense_layer_2)
output = Dense(np.transpose(scaled_y).shape[1], activation='softmax')(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [61]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48)]              0         
_________________________________________________________________
dense (Dense)                (None, 15)                735       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, verbose=1, validation_split=0.2)

Epoch 1/500
409/409 [==============================] - 1s 2ms/step - loss: 0.2302 - acc: 0.9223 - val_loss: 0.2429 - val_acc: 0.9169
Epoch 2/500
409/409 [==============================] - 1s 2ms/step - loss: 0.2352 - acc: 0.9162 - val_loss: 0.2117 - val_acc: 0.9364
Epoch 3/500
409/409 [==============================] - 1s 2ms/step - loss: 0.1922 - acc: 0.9229 - val_loss: 0.1957 - val_acc: 0.9291
Epoch 4/500
409/409 [==============================] - 1s 2ms/step - loss: 0.2035 - acc: 0.9278 - val_loss: 0.2512 - val_acc: 0.9022
Epoch 5/500
409/409 [==============================] - 1s 2ms/step - loss: 0.1827 - acc: 0.9284 - val_loss: 0.2913 - val_acc: 0.8998
Epoch 6/500
409/409 [==============================] - 1s 2ms/step - loss: 0.1814 - acc: 0.9278 - val_loss: 0.2886 - val_acc: 0.8949
Epoch 7/500
409/409 [==============================] - 1s 2ms/step - loss: 0.1761 - acc: 0.9290 - val_loss: 0.2614 - val_acc: 0.9022
Epoch 8/500
409/409 [==============================] - 1s 2ms/step - 

In [69]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

16/16 [==============================] - 0s 2ms/step - loss: 0.1553 - acc: 0.9472
Test Score: 0.15533865988254547
Test Accuracy: 0.9471624493598938
